# imports

In [1]:
from pathlib import Path

In [31]:
import bs4

In [2]:
from bs4 import BeautifulSoup as BS

In [69]:
from dataclasses import dataclass

In [70]:
from enum import Enum

# Récupération du contenu html

On a **manuellement** sauvegarder les 7 premières pages correspondant aux résultats de la recherche *python* sur [amazon.fr](www.amazon.fr)

In [3]:
racine = Path(".").resolve()
print(racine)

C:\Users\perrollaz\Documents


In [5]:
for contenu in racine.glob("*"):
    print(contenu)

C:\Users\perrollaz\Documents\.ipynb_checkpoints
C:\Users\perrollaz\Documents\02.ipynb
C:\Users\perrollaz\Documents\03.ipynb
C:\Users\perrollaz\Documents\afterSentDocuments
C:\Users\perrollaz\Documents\desktop.ini
C:\Users\perrollaz\Documents\Ma musique
C:\Users\perrollaz\Documents\Mes images
C:\Users\perrollaz\Documents\Mes vidéos
C:\Users\perrollaz\Documents\pages


In [6]:
pages = racine / "pages"
print(pages)

C:\Users\perrollaz\Documents\pages


In [10]:
for chemin in pages.glob("*.htm"):
    print(chemin)

C:\Users\perrollaz\Documents\pages\01.htm
C:\Users\perrollaz\Documents\pages\02.htm
C:\Users\perrollaz\Documents\pages\03.htm
C:\Users\perrollaz\Documents\pages\04.htm
C:\Users\perrollaz\Documents\pages\05.htm
C:\Users\perrollaz\Documents\pages\06.htm
C:\Users\perrollaz\Documents\pages\07.htm


In [12]:
htmls = list()
for chemin in pages.glob("*.htm"):
    htmls.append(chemin.read_text(encoding="utf8"))

# Exploitation de la première page html

## Exercice

En utilisant les outils de développement du navigateur et `bs4`, récupérer la liste des éléments représentant des livres dans la première page.

In [23]:
soupe = BS(htmls[0])

In [24]:
result_items = soupe.find_all(name="div", attrs={"class": "s-result-item"})

In [25]:
len(result_items)

69

**ATTENTION** on voit visuellement qu'il n'y a que 60 éléments à récupérer dans la page, donc il y a des intrus à identifier.

In [26]:
result_item = result_items[1]

In [45]:
print(result_item.prettify())

<div class="sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20" data-asin="B0BHRB3MPR" data-cel-widget="search_result_1" data-component-id="19" data-component-type="s-search-result" data-index="2" data-uuid="d068c86e-6b48-487b-93ae-0c3b2d26e4c1">
 <div class="sg-col-inner">
  <div cel_widget_id="MAIN-SEARCH_RESULTS-2" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-cel-widget="MAIN-SEARCH_RESULTS-2" data-csa-c-id="bxnszw-3mpitn-34b5kj-di4ojw" data-csa-c-item-id="amzn1.asin.1.B0BHRB3MPR" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render="">
   <div class="rush-component s-expand-height" data-component-id="20" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-eu.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTra

In [28]:
result_item.find_all(name="span", attrs={"class": "a-text-normal"})

[<span class="a-size-base-plus a-color-base a-text-normal">Python pour débutants: Programmer un réseau neuronal en 7 jours</span>]

In [29]:
result_item.find_all(name="span", attrs={"class": "a-price-whole"})

[<span class="a-price-whole">16,99</span>]

In [34]:
def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    titre, = element.find_all(name="span", attrs={"class": "a-text-normal"})
    prix, = element.find_all(name="span", attrs={"class": "a-price-whole"})
    return (titre.text, prix.text)
    

In [35]:
recup_titre_prix(result_item)

('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')

In [41]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ValueError:
        pass

In [42]:
# il y a normalement 60 livres dans la page
len(extractions)

59

In [44]:
# Inspection visuelle
for extraction in extractions:
    print(extraction)

('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
("PYTHON,\n Guide complet de la programmation orientée objet pour débutants: \nDevenez un développeur en créant une IA avec l'apprentissage automatique\n et un site web en 10 étapes. Plus de 200 exercices.", '31,64')
("Apprendre\n Python: Un Cours Accéléré sur la Programmation Python et Comment \nCommencer à l’utiliser pour Coder. Apprenez les Bases de Machine \nLearning et de l'analyse de Données", '16,87')
("PYTHON: L'introduction et les fondements pour apprendre à programmer en python en 7 jours", '12,90')
('Apprendre à programmer avec Python 3', '32,40')
('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
("J'apprends à programmer avec Python", '12,95')
('Apprendre à dessiner pour enfants', '6,99')
("100\n Exercices Python pour s'entrainer: Un ensemble d'exercices avec \ndifférents niveaux de complexité | Débutant - Intermédiaire - Avancé | \nExercices corrigés pour tous les niveau

## Exercice

Ajuster la recherche précédente pour récupérer exactement les résultats voulus.

In [46]:
for extraction in extractions:
    if "python" not in extraction[0].lower():
        print(extraction)

('Apprendre à dessiner pour enfants', '6,99')
("C++: Introduction et principes fondamentaux du langage de programmation orienté. L'objet le plus célèbre du monde.", '12,90')
('Les algorithmes, c’est plus simple avec un dessin !', '19,90')
('Apprenons à tracer : Lignes Formes Lettres : Cahier d’activités pour enfants : Âges 3 et +', '6,99')


In [63]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(name="span", attrs={"class": "a-text-normal"})
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(name="span", attrs={"class": "a-price-whole"})
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [64]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ParsingError:
        pass

In [66]:
len(extractions)

55

# Assemblage

In [ ]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [67]:
extractions = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        try:
            extractions.append(recup_titre_prix(item))
        except ParsingError:
            pass

In [68]:
len(extractions)

312

## Exercice

Enrichir le code pour extraire quand c'est possible:
- les auteurs
- l'évalution moyenn
- le nombres d'évaluations
- broché ou relié ou...
- sponsorisé ou pas?

In [71]:
class Genre(Enum):
    RELIE = "relié"
    BROCHE = "broché"
    ELECTRONIQUE = "électronique"

In [72]:
# A sérialiser avec pyserde vers du json
@dataclass
class Article:
    titre: str
    prix: str
    auteurs: str
    evaluation: float
    nombre_evaluations: int
    genre: Genre
    sponsorise: bool

In [73]:
def recupere_titre(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return titre

In [74]:
def recupere_prix(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    return prix

In [ ]:
def recupere_auteurs(element: bs4.element.Tag) -> str:
    ...

def recupere_evaluation(element: bs4.element.Tag) -> float:
    ...
    
def recupere_nombre_evaluations(element: bs4.element.Tag) -> int:
    ...
    
def recupere_genre(element: bs4.element.Tag) -> Genre:
    ...


In [ ]:
def recupere_sponsorise(element: bs4.element.Tag) -> bool:
    spans = element.find_all(
        name="span",
        attrs={"class": "a-color-base"}
    )
    for span in spans:
        if span.text == "Sponsorisé":
            return True
    return False

In [ ]:
def genere_article(element: bs4.element.Tag) -> Article:
    ...

In [ ]:
articles: list[Article] = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        articles.append(genere_article(item))
        


## Exercice

Sérialiser vers json `articles`.